# MWE of PCH: Periodic deblurring problem

First import packages required

In [ ]:
import sys

sys.path.append("..") 
import numpy as np
from scipy.sparse import diags
import matplotlib.pyplot as plt
import inspect

# Set rng seed 
np.random.seed(0)

Then import tools needed from CUQIpy:

In [ ]:
from cuqi.testproblem import Deconvolution
from cuqi.model import LinearModel
from cuqi.distribution import Gaussian, Laplace_diff, Cauchy_diff
from cuqi.sampler import CWMH
from cuqi.problem import BayesianProblem
from cuqi.samples import Samples, Data

## 1. Loading the Deconvolution problem

We want to experiment with the Deconvolution test problem; we first take a look at how to call it:

In [ ]:
help(Deconvolution)

Specify parameters and create Deconvolution TestProblem:

In [ ]:
dim = 128
kernel = ["Gauss","Sinc","vonMises"]
phantom = ["Gauss","Sinc","vonMises","Square","Hat","Bumps","DerivGauss"]
noise_type = ["Gaussian","ScaledGaussian"]
noise_std = 0.05

# Test problem
prob = Deconvolution(
    dim = dim,
    kernel=kernel[0],
    phantom=phantom[3],
    noise_type=noise_type[0],
    noise_std = noise_std
)

A CUQI TestProblem already contains everything apart from a prior to do UQ, so we could simply specify a prior and then run the UQ method directly on the test problem. This is because TestProblem is a subclass of BayesianProblem, so the UQ method is inherited. But, here we will "unpack" the main ingredients (the linear operator and the original clean signal) and build our problem from scratch.

In [ ]:
A = prob.model.get_matrix()
phantom = prob.exactSolution     # We may later return and generate phantom as a sample from a prior distribution.

We can show the matrix and the "original clean signal" directly using python matplotlib plotting methods:

In [ ]:
plt.figure()
plt.imshow(A)
plt.figure()
plt.plot(phantom)

## 2. Set up problem in CUQIpy with IID Gaussian prior

We first set up a CUQI LinearModel:

In [ ]:
# Define as linear model
model = LinearModel(A)

In [ ]:
phantomD = Data(phantom,  model.domain_geometry)

In [ ]:
type(phantomD)

In [ ]:
phantomD.plot(linestyle='--')

With the CUQI model we can for example apply it to the clean signal and obtain clean data and plot it:

In [ ]:
data_clean = model(phantom)
plt.plot(data_clean)

In [ ]:
data_cleanD = model(phantomD)
data_cleanD.plot()

We can use the geometries of the domain and range of the model for the dedicated plot type (for a simple 1D signal like this, the default Continuous1D geometry is automatically employed and its plotting method simply add the xlabel)

In [ ]:
model.domain_geometry.plot(phantom)
plt.figure()
model.range_geometry.plot(data_clean)

In [ ]:
phantomD.plot()
plt.figure()
data_cleanD.plot()

Define likelihood as Gaussian with model as mean and i.i.d. noise with 0.05 std.  Note how the CUQIpy object "model" is given as the mean. Thus likelihood is a conditional distribution, which can only be sampled once the conditioned parameter is specified.

In [ ]:
noise_std = 0.05
likelihood = Gaussian(model, noise_std, np.eye(dim))

Generate and plot samples of noise by conditioning on the zero image:

In [ ]:
likelihood(x=np.zeros(dim)).sample(5).plot()
plt.title('Noise samples'); plt.show()

In the same way, by conditioning on the true phantom, we generate (noisy) samples of simulated data:

In [ ]:
likelihood(x=phantom).sample(5).plot()
plt.title('Simulated data'); plt.show()

Now generate single realization of data to be used in inverse problem:

In [ ]:
data = likelihood(x=phantom).sample()

Specify the prior distribution and generate and plot 5 samples:

In [ ]:
prior_std = 0.2
prior = Gaussian(np.zeros(dim), prior_std, np.eye(dim))

# Plot samples of prior
prior.sample(5).plot()
plt.title('Realizations from prior'); plt.show()

Combine likelihood, prior and the observed data into "BayesianProblem" object representing the inverse problem. No solving yet.

In [ ]:
IP = BayesianProblem(likelihood, prior, data)

## 3. Solving the inverse problem

We can ask for the MAP estimate:  (for small fully Gaussian problems we use analytical expression; for more complicated problems support is underway using numerical optimization. Prototype support implemented at hackathon using gradients of logpdf and LBFGS from scipy)

In [ ]:
x_MAP = IP.MAP() 

# Plot
#plt.plot(phantom,'.-')
#plt.plot(x_MAP,'.-')
phantomD.plot()
x_MAP.plot()
plt.title("MAP estimate")
plt.legend(["Exact","MAP"])
plt.show()

Instead of MAP, we can sample the posterior:

In [ ]:
Ns = 5000   # Number of samples
result = IP.sample_posterior(Ns)

The output is a cuqi.Samples object containing the samples and various utilities such as plotting methods:

In [ ]:
type(result)

For example we can plot confidence interval around the exact:

In [ ]:
result.plot_ci(95, exact=phantom)

And the posterior sample std:

In [ ]:
result.plot_std()

As well as chains of the individual parameters, here three shown:

In [ ]:
idx = [20,55,60]
result.plot_chain(idx)
plt.legend(idx)

Diagnostics of the sampling is ongoing work, some information is available currently in result:

In [ ]:
result.diagnostics()

## 4. Change to a Gaussian with correlation

Define correlation matrix where 30 closest neighbours are correlated

In [ ]:
l = 30
corr = np.linspace(0,1,int(l/2)+1)
corr = np.hstack((corr,np.flipud(corr[:-1])))
indexes = np.linspace(-l/2,l/2,l+1,dtype=int)
corrmat = diags(corr, indexes, shape=(dim,dim)).toarray()

plt.imshow(corrmat)

Set new prior and plot a few samples:

In [ ]:
IP.prior = Gaussian(np.zeros(dim), prior_std, corrmat)

# Plot samples from prior
IP.prior.sample(5).plot() 
plt.title('Realizations from prior'); plt.show()

We have already updated the "prior" attribute of the IP, so we are ready to solve for the MAP and sample:

In [ ]:
x_MAP_corr = IP.MAP()

# Plot
phantomD.plot()
x_MAP_corr.plot()
plt.title("Map estimate")
plt.legend(["Exact","MAP"])
plt.show()

In [ ]:
result_corr = IP.sample_posterior(Ns)

# plot mean + 95% of samples
result_corr.plot_ci(95, exact=phantom)

## 5. Change to a Cauchy prior

The correlated Gaussian gives a narrow confidence interval so was better than the IID Gaussian, but still cannot reproduce the sharp edges. We try a Cauchy prior.

In [ ]:
scale = 2/dim
IP.prior = Cauchy_diff(np.zeros(dim), scale, 'neumann')

MAP for Cauchy is not yet implemented so `x_MAP_cau = IP.MAP()` would fail, but we can sample the posterior. This uses a different sampling method than Gaussian and this is slower.

In [ ]:
result_cau = IP.sample_posterior(Ns)

Plot mean + 95% of samples

In [ ]:
result_cau.plot_ci(95, exact=phantom)

In [ ]:
result_cau.plot_std()

## 6. Change to Laplace prior

Cauchy is known to be very edge-preserving but difficult to work with for example sample. Another edge-preserving prior is Laplace which typically is more well-behaved but not quite as strongly edge-preserving

In [ ]:
# Set up Laplace prior
loc = np.zeros(dim)
delta = 0.5
scale = delta*1/dim
prior = Laplace_diff(loc,scale,'zero')

Laplace prior is not yet supported by the BayesianProblem.sample_posterior, but we can manually sample the posterior for example using the Componentwise Metropolis-Hastings (CWMH) sampler. Currently this is set up by specifying target and proposal FUNCTIONS but is in the process of being updated to specify using CUQIpy DISTRIBUTIONS.

In [ ]:
# Target and proposal
def target(x): return IP.likelihood.logpdf(IP.data,x)+prior.logpdf(x)
def proposal(x,scale): return np.random.normal(x,scale)

# Parameters for sampler
scale = 0.05*np.ones(dim)
x0 = 0.5*np.ones(dim)

# Define sampler (Component-Wise Metroplis-Hastings)
MCMC = CWMH(target, proposal, scale, x0)

# Burn-in
Nb = int(0.2*Ns)   

Run sampler (with adaptive parameter selection):

In [ ]:
x_s, out2, out3 = MCMC.sample_adapt(Ns,Nb)

This particular samples does not yet produce a Samples object, just the numpy array with samples in columns

In [ ]:
x_s

But we can manually create the Samples object for now:

In [ ]:
result_lap = Samples(x_s)

Then we can use the plotting methods such as of the confidence interval:

In [ ]:
result_lap.plot_ci(95, exact=phantom)

Or std plot:

In [ ]:
result_lap.plot_std()

## Some ideas to try out (warning: not tested!)
1. Change phantom to another of the options provided by the TestProblem.
2. Change phantom to be a sample from Gaussian, then do UQ with correct and incorrect choices of prior.
3. Play with parameter of priors for example Cauchy and Laplace to see if a better solution can be obtained.
3. Change noise/likelihood to scaledGaussian